In [25]:
import QuantLib as ql

# Assuming today's date is the settlement date
settlement_date = ql.Date(2, 2, 2024)
ql.Settings.instance().evaluationDate = settlement_date


# Spot rates provided

spot_rates = [
    5.49,
    5.51,
]  # in percent


# Corresponding tenors in months and years for deposits and bonds
deposit_tenors_months = [1, 3, 6]
bond_tenors_years = [1, 2]

# Convert annual spot rates to QuantLib QuoteHandle objects and adjust for percentage
spot_rate_handles = [
    ql.QuoteHandle(ql.SimpleQuote(rate / 100.0)) for rate in spot_rates
]

# Define day count and calendar for deposits
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count_deposit = ql.Actual360()

# Deposit rate helpers
deposit_helpers = [
    ql.DepositRateHelper(
        spot_rate_handles[i],
        ql.Period(tenors, ql.Months),
        2,  # fixing days
        calendar,
        ql.ModifiedFollowing,
        False,
        day_count_deposit,
    )
    for i, tenors in enumerate(deposit_tenors_months)
]

# Construct the curve
curve = ql.PiecewiseLogCubicDiscount(
    settlement_date, deposit_helpers, ql.ActualActual(ql.ActualActual.ISDA)
)

date, rates = zip(*curve.nodes())
rates

IndexError: list index out of range

In [35]:

# Day count for bonds
day_count_bond = ql.ActualActual(ql.ActualActual.ISDA)

prices = [99, 99.5]
# Bond rate helpers
bond_helpers = []
for i, tenors in enumerate(
    bond_tenors_years, start=0
):  # Starting from the 4th element in spot_rates
    maturity_date = settlement_date + ql.Period(tenors, ql.Years)
    schedule = ql.Schedule(
        settlement_date,
        maturity_date,
        ql.Period(ql.Annual),
        calendar,
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False,
    )
    bond_helpers.append(
        ql.FixedRateBondHelper(
            ql.QuoteHandle(ql.SimpleQuote(prices[i])),
            3,  # settlement days
            100.0,  # face amount
            schedule,
            [spot_rates[i] / 100.0],  # coupon rate
            day_count_bond,
            ql.ModifiedFollowing,
            100.0,
        )  # redemption
    )


In [31]:
len(bond_helpers)

2

In [36]:

# Combine deposit and bond helpers
rate_helpers = deposit_helpers + bond_helpers

# Construct the curve
curve = ql.PiecewiseLogCubicDiscount(
    settlement_date, bond_helpers, ql.ActualActual(ql.ActualActual.ISDA)
)

date, rates = zip(*curve.nodes())

In [37]:
date, rates

((Date(2,2,2024), Date(3,2,2025), Date(2,2,2026)),
 (1.0, 0.9383150856835977, 0.8938731111995456))

In [ ]:

# Extract zero rates for months 1-360
zero_rates = []
for month in range(1, 361):
    date = settlement_date + ql.Period(month, ql.Months)
    yrs = curve.dayCounter().yearFraction(settlement_date, date)
    zero_rate = curve.zeroRate(yrs, ql.Compounded, ql.Annual).rate()
    zero_rates.append(zero_rate)

# Print some of the zero rates
print("Zero Rates for the first 12 months:")
for month, rate in enumerate(zero_rates[:12], start=1):
    print(f"Month {month}: {rate*100:.2f}%")

In [44]:
import QuantLib as ql

# Spot rates provided
spot_rates = [
    5.49,
    5.51,
    5.43,
    5.42,
    5.22,
    4.81,
    4.36,
    4.14,
    3.99,
    4.02,
    4.03,
    4.33,
    4.22,
]  # in percent

# Corresponding tenors in months and years for deposits and bonds
deposit_tenors_months = [1, 3, 6]
bond_tenors_years = [1, 2, 3, 5, 7, 10, 20, 30]

# Convert annual spot rates to QuantLib QuoteHandle objects and adjust for percentage
spot_rate_handles = [
    ql.QuoteHandle(ql.SimpleQuote(rate / 100.0)) for rate in spot_rates
]

# Assuming today's date is the settlement date
settlement_date = ql.Date(2, 2, 2024)
ql.Settings.instance().evaluationDate = settlement_date

# Define day count and calendar for deposits
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count_deposit = ql.Actual360()

# Deposit rate helpers
deposit_helpers = [
    ql.DepositRateHelper(
        spot_rate_handles[i],
        ql.Period(tenors, ql.Months),
        2,  # fixing days
        calendar,
        ql.ModifiedFollowing,
        False,
        day_count_deposit,
    )
    for i, tenors in enumerate(deposit_tenors_months)
]

# Day count for bonds
day_count_bond = ql.ActualActual(ql.ActualActual.ISDA)

prices= [
    ql.QuoteHandle(ql.SimpleQuote(rate+90)) for rate in spot_rates
]
# Bond rate helpers
bond_helpers = []
for i, tenors in enumerate(
    bond_tenors_years, start=3
):  # Starting from the 4th element in spot_rates
    maturity_date = settlement_date + ql.Period(tenors, ql.Years)
    schedule = ql.Schedule(
        settlement_date,
        maturity_date,
        ql.Period(ql.Annual),
        calendar,
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False,
    )
    bond_helpers.append(
        ql.FixedRateBondHelper(
            prices[i],
            3,  # settlement days
            100.0,  # face amount
            schedule,
            [spot_rates[i] / 100.0],  # coupon rate
            day_count_bond,
            ql.ModifiedFollowing,
            100.0,
        )  # redemption
    )

# Combine deposit and bond helpers
rate_helpers = deposit_helpers + bond_helpers

# Construct the curve
curve = ql.PiecewiseLogCubicDiscount(
    settlement_date, rate_helpers, ql.ActualActual(ql.ActualActual.ISDA)
)

# Extract zero rates for months 1-360
zero_rates = []
for month in range(1, 361):
    date = settlement_date + ql.Period(month, ql.Months)
    yrs = curve.dayCounter().yearFraction(settlement_date, date)
    zero_rate = curve.zeroRate(yrs, ql.Compounded, ql.Annual).rate()
    zero_rates.append(zero_rate)

# Print some of the zero rates
print("Zero Rates for the first 12 months:")
for month, rate in enumerate(zero_rates[:12], start=1):
    print(f"Month {month}: {rate*100:.2f}%")

Zero Rates for the first 12 months:
Month 1: 5.69%
Month 2: 5.84%
Month 3: 5.75%
Month 4: 5.43%
Month 5: 5.27%
Month 6: 5.53%
Month 7: 6.27%
Month 8: 7.23%
Month 9: 8.26%
Month 10: 9.17%
Month 11: 9.92%
Month 12: 10.43%
